In [6]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer
import itertools

# HW2:  Модель Word2vec
### Выполнили:  

* Булгаков Дмитрий

* Тефикова Алие

### Группа ИАД-2

## 0. Loading data

Составьте достаточно большую коллекцию текстов на любом языке или ис-пользуйте тексты отсюда:<br>
https://www.kaggle.com/c/word2vec-nlp-tutorial/data.

In [7]:
text_dataset = pd.read_csv('data/unlabeledTrainData.csv', sep=';', encoding = 'latin1')

In [8]:
text_dataset.head(n=5)

,id,review
0,9999_0,"Watching Time Chasers, it obvious that it was ..."
1,45057_0,I saw this film about 20 years ago and remembe...
2,15561_0,"Minor SpoilersIn New York, Joan Barnard (Elvir..."
3,7161_0,I went to see this film with a great deal of e...
4,43971_0,"Yes, I agree with everyone on this site this m..."


In [9]:
def split_dataset_into_list_sentences(df, size=20000):
    df_sentences_list = []
    for i in range(0, size):
        df_sentences_list.append(sent_tokenize(df.review.values[i]))
    return df_sentences_list

In [10]:
def flatten_list_of_lists(list_of_lists):
    return list(itertools.chain.from_iterable(list_of_lists))

In [11]:
def split_sentence_into_words_list(sentence):
    tokenizer = RegexpTokenizer(r'\w+')
    return [x.lower() for x in tokenizer.tokenize(sentence)]

In [12]:
def split_list_of_sentences_into_words_list(list_sentences):
    list_sentences_words = []
    for sentence in list_sentences:
        list_sentences_words.append(split_sentence_into_words_list(sentence))
    return list_sentences_words

In [13]:
def process_dataset(dataset):
    list_sentences = split_dataset_into_list_sentences(dataset)
    list_sentences = flatten_list_of_lists(list_sentences)
    list_sentences_words = split_list_of_sentences_into_words_list(list_sentences)
    return list_sentences_words

In [14]:
%time
processed_dataset = process_dataset(text_dataset)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs


## 1. Fitting Word2Vec model

Обучите модель word2vec. Оцените время обучения модели,используя модуль time. Есть два варианта обучения модели: по отзывам целиком и с уче-там границ предложений. В принципе, погрешностью, которая возникает в первом случае можно пренебречь, но если вы хотите учитывать границы предложений, то можно использовать sent_tokenize из nltk.tokenize

In [15]:
%time
model = Word2Vec(processed_dataset)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.96 µs


## 2. Looking for similar words

Примеры практически безошибочного (во всяком случае для ТОП-3) определения синонимов с помощью most_similar для следующих слов: 

**Amazing**

In [199]:
model.most_similar(positive = ['amazing'], topn = 3)

[('incredible', 0.8192795515060425),
 ('awesome', 0.8182525634765625),
 ('outstanding', 0.7644368410110474)]

**Stunning**

In [197]:
model.most_similar(positive = ['stunning'], topn = 3)

[('superb', 0.8590769171714783),
 ('breathtaking', 0.8518157601356506),
 ('magnificent', 0.8199763894081116)]

**Great**

In [260]:
model.most_similar(positive = ['great'], topn = 3)

[('wonderful', 0.8114394545555115),
 ('fine', 0.752615213394165),
 ('fantastic', 0.7393239736557007)]

**Thrilling**

In [266]:
model.most_similar(positive = ['thrilling'], topn = 3)

[('suspenseful', 0.8817949295043945),
 ('sentimental', 0.8348844051361084),
 ('unpredictable', 0.833819568157196)]

**Huge**

In [379]:
model.most_similar(positive = ['huge'], topn = 3)

[('big', 0.7379018068313599),
 ('massive', 0.5902572274208069),
 ('large', 0.5687769651412964)]

**Recommendation**

In [293]:
model.most_similar(positive = ['recommendation'], topn = 3)

[('advice', 0.8001496195793152),
 ('suggestion', 0.7946276068687439),
 ('drift', 0.7682980298995972)]

Примеры, где синонимами являются только ТОП-**2** похожих слова: 

**Film**

In [193]:
model.most_similar(positive = ['film'], topn = 3)

[('movie', 0.9304608106613159),
 ('picture', 0.7950036525726318),
 ('documentary', 0.7226828932762146)]

**Channel**

In [343]:
model.most_similar(positive = ['channel'], topn = 3)

[('cable', 0.8390822410583496),
 ('network', 0.7997230291366577),
 ('bbc', 0.7517694234848022)]

**Place**

In [361]:
model.most_similar(positive = ['place'], topn = 3)

[('area', 0.6078256368637085),
 ('town', 0.5167050957679749),
 ('country', 0.5166371464729309)]

Как видно, и documentry, и bbc, и country является похожими по смыслу словами, но не синонимами, так как это скорее слова, которые включается в большое множество film, channel и place, т.е. являются их подмножествами, а не эквивалентны им.

**Kind**

In [191]:
model.most_similar(positive = ['kind'], topn = 3)

[('sort', 0.8789204359054565),
 ('type', 0.6961961388587952),
 ('course', 0.5455812215805054)]

В случае с kind, опять же синонимами являются только первые два слова, что также видно и по значению ***похожести????*** или как сказать

**Lucky**

In [198]:
model.most_similar(positive = ['lucky'], topn = 3)

[('fortunate', 0.7852784991264343),
 ('upset', 0.6359814405441284),
 ('willing', 0.5791153907775879)]

В данном примере синонимом является уже только первое слово, что говорит о том, что most_similar не всегда подходит для определения похожих слов с точки зрения их лексического значения. Также это будет подтверждено следующими примерами.

Случаи, когда самым схожим словом является - **антоним** (love - hate, day - night):

**Love**

In [516]:
model.most_similar(positive = ['love'], topn = 3)

[('hate', 0.6232635378837585),
 ('loved', 0.511492908000946),
 ('true', 0.46797704696655273)]

**Day**

In [515]:
model.most_similar(positive = ['day'], topn = 3)

[('night', 0.6791311502456665),
 ('week', 0.6550660133361816),
 ('morning', 0.617975652217865)]

Можно заметить, что большую часть наших удачных примеров составляют прилагательные. Это связано с тем, что именно для прилагательных модель выдает лучшие результаты, то есть точнее определяет синонимы. Скорее всего, это связано с тем, что при написании рецензий используется огромное количество всевозможных прилагательных, что дает модели возможность лучше обучиться. Благодаря этому, для данного текста most_similar действительно подходит для определения синонимов слов, точность результатов которого значительно повышается, если часть речи проверяемого слова - прилагательное.  

## 3. Looking for associations

Приведите 5-10 примеров использования .most_similar для определения ассоциаций (А к Б, как В к?). Корректно ли най- дены ассоциации?

In [533]:
model.most_similar(positive = ['hot'], topn = 3)

[('blond', 0.7752785086631775),
 ('blonde', 0.7552344799041748),
 ('naked', 0.7540833950042725)]

In [417]:
model.most_similar(positive = ['america'], topn = 3)

[('japan', 0.8229783177375793),
 ('germany', 0.8226988911628723),
 ('europe', 0.8164801001548767)]

In [412]:
model.most_similar(positive = ['car', 'accident'], topn = 3)

[('plane', 0.809522271156311),
 ('crash', 0.7584969997406006),
 ('truck', 0.7568835020065308)]

In [418]:
model.most_similar(positive = ['mother', 'father'], topn = 3)

[('husband', 0.9241902828216553),
 ('son', 0.9222783446311951),
 ('daughter', 0.9172472953796387)]

In [421]:
model.most_similar(positive = ['work', 'job'], topn = 3)

[('talents', 0.5569989681243896),
 ('career', 0.5482080578804016),
 ('justice', 0.5450432300567627)]

In [431]:
model.most_similar(positive = ['war', 'death', 'soldiers'], topn = 3)

[('civil', 0.7497152090072632),
 ('nazi', 0.7184252142906189),
 ('destruction', 0.7139583826065063)]

In [444]:
model.most_similar(positive = ['crime', 'blood', 'victim'], topn = 3)

[('sword', 0.7973902225494385),
 ('murder', 0.6980674266815186),
 ('disease', 0.6750417351722717)]

In [492]:
model.most_similar(positive = ['film', 'funny'], topn = 3)

[('movie', 0.8055394887924194),
 ('entertaining', 0.6449971199035645),
 ('comedy', 0.6269345283508301)]

## 4. Looking for inappropriate words

Приведите 5-10 примеров использования .doesnt_match для определения лишнего слова. Корректно ли найдены лишние слова?

Model finds words that out of context

In [17]:
model.doesnt_match("pizza food popcorn salad fries character".split())

'character'

In [18]:
model.doesnt_match("movie film cinema hero actor singer".split())

'singer'

In [19]:
model.doesnt_match("schindlers jews fun pain prison kill".split())

'fun'

In [20]:
model.doesnt_match("love joy like enjoy excite boring ".split())

'boring'

And even distinguishes male from female names

In [21]:
model.doesnt_match("mary joan caroline john malisa keith".split())

'john'

## 5. Looking for 

Пара слов, для которых не выполняется свойство коммутативности

In [395]:
model.most_similar(positive = ['groove'], topn = 3)

[('dashiell', 0.8891346454620361),
 ('hurst', 0.8772737979888916),
 ('akroyd', 0.8757452964782715)]

In [396]:
model.most_similar(positive = ['hurst'], topn = 3)

[('judas', 0.8848900198936462),
 ('chori', 0.8839111328125),
 ('adama', 0.8822944760322571)]

Как видно, Hurst входит в ТОП-3 похожих слов для Groove, а вот Groove не в ходит в ТОП-3 Hurst, зато в него входят имена, из чего делаем вывод, что в контексте наших данных Hurst встречалось чаще всего как фамилия, нежели как слово "бугор". 

Пара слов, для которых выполняется свойство коммутативности

In [401]:
model.most_similar(positive = ['lovely'], topn = 3)

[('gorgeous', 0.8295367956161499),
 ('beautiful', 0.7924778461456299),
 ('sexy', 0.768959641456604)]

In [403]:
model.most_similar(positive = ['gorgeous'], topn = 3)

[('lovely', 0.8295369148254395),
 ('beautiful', 0.7663096189498901),
 ('sexy', 0.765828549861908)]

Как уже было замечено ранее, при написании рецензий авторы чаще всего используют прилагательные, поэтому эта часть речи, а особенно наиболее популярные слова в больших случаях обладает свойством коммутативности, что подтверждается нашим примером с lovely и gorgeous.